# Prepare complete dataset
When comparing two datasets fetched from SpaceX API and Falcon 9 Wikipedia page, I can see that there are valuable data in both csv files. But these files cannot be merged together.
So, I have to collect data from both csv file and prepare a complete dataset.

In this Notebook I combine data from both csv files and prepare a complete and final dataset.

In [1]:
import pandas as pd
import numpy as np
import csv, sqlite3, sqlalchemy

In [2]:
df_api = pd.read_csv("spacex_falcon9.csv")
df_api.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,8191.07911,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.00000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.00000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.00000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.00000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.00000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.00000,ISS,CCSFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.00000,LEO,CCSFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.00000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.00000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


In [3]:
df_wiki = pd.read_csv("spacex_falcon9_wiki.csv")
df_wiki.head(10)

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCSFS,Dragon Spacecraft Qualification Unit,N,LEO,SpaceX,Success,F9 v1.0,Failure,2010-06-04,18:45:00
1,2,CCSFS,SpaceX COTS Demo Flight 1,U,LEO,NASA,Success,F9 v1.0,Failure,2010-12-08,15:43:00
2,3,CCSFS,SpaceX COTS Demo Flight 2,525 kg,LEO,NASA,Success,F9 v1.0,No attempt,2012-05-22,07:44:00
3,4,CCSFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,F9 v1.0,No attempt,2012-10-08,00:35:00
4,5,CCSFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,F9 v1.0,No attempt,2013-03-01,15:10:00
5,6,VSFB,CASSIOPE,500 kg,Polar orbit,MDA,Success,F9 v1.1,Uncontrolled,2013-09-29,16:00:00
6,7,CCSFS,SES-8,"3,170 kg",GTO,SES,Success,F9 v1.1,No attempt,2013-12-03,22:41:00
7,8,CCSFS,Thaicom 6,"3,325 kg",GTO,Thaicom,Success,F9 v1.1,No attempt,2014-01-06,22:06:00
8,9,CCSFS,SpaceX CRS-3,"2,296 kg",LEO,NASA,Success,F9 v1.1,Controlled,2014-04-18,19:25:00
9,10,CCSFS,Orbcomm-OG2,"1,316 kg",LEO,Orbcomm,Success,F9 v1.1,Controlled,2014-07-14,15:15:00


In [4]:
df_api.shape

(168, 17)

In [5]:
df_wiki.shape

(243, 11)

<div class="alert alert-info">
    <h4>Observations:</h4>
    <ol>
        <li>There are more observations from Wikipedia page than SpaceX API</li>
    </ol>
</div>

I will take Wikipedia dataset as the base dataset and fill it up with the information taken from SpaceX API dataset.

Launch sites from API

In [6]:
df_api['LaunchSite'].value_counts()

CCSFS SLC 40    93
KSC LC 39A      49
VAFB SLC 4E     26
Name: LaunchSite, dtype: int64

#### Replace Launch Sites with proper names

In [7]:
launch_site_dict = {
    'CCSFS': 'CCSFS SLC 40',
    'KSC': 'KSC LC 39A',
    'VSFB': 'VAFB SLC 4E',
    'VAFB': 'VAFB SLC 4E'
}

df_wiki['Launch site'].replace(launch_site_dict, inplace=True)

In [8]:
df_wiki['Launch site'].unique()

array(['CCSFS SLC 40', 'VAFB SLC 4E', 'KSC LC 39A'], dtype=object)

#### Correct the *Payload mass* values
- Remove the unit kg and the comma
- Replace char values with numbers
- Set float type

In [10]:
corrected_payload_mass_list = []

for payload_mass in df_wiki['Payload mass']:
    new_pm = payload_mass.strip(' kg').replace(',', '')
    if not new_pm.isdigit():
        new_pm = 0
    corrected_payload_mass_list.append(new_pm)
    
df_wiki['Payload mass'] = corrected_payload_mass_list

In [11]:
df_wiki.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCSFS SLC 40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,F9 v1.0,Failure,2010-06-04,18:45:00
1,2,CCSFS SLC 40,SpaceX COTS Demo Flight 1,0,LEO,NASA,Success,F9 v1.0,Failure,2010-12-08,15:43:00
2,3,CCSFS SLC 40,SpaceX COTS Demo Flight 2,525,LEO,NASA,Success,F9 v1.0,No attempt,2012-05-22,07:44:00
3,4,CCSFS SLC 40,SpaceX CRS-1,4700,LEO,NASA,Success,F9 v1.0,No attempt,2012-10-08,00:35:00
4,5,CCSFS SLC 40,SpaceX CRS-2,4877,LEO,NASA,Success,F9 v1.0,No attempt,2013-03-01,15:10:00


In [12]:
df_wiki['Payload mass'] = df_wiki['Payload mass'].astype('float')

In [13]:
df_wiki.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCSFS SLC 40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,F9 v1.0,Failure,2010-06-04,18:45:00
1,2,CCSFS SLC 40,SpaceX COTS Demo Flight 1,0.0,LEO,NASA,Success,F9 v1.0,Failure,2010-12-08,15:43:00
2,3,CCSFS SLC 40,SpaceX COTS Demo Flight 2,525.0,LEO,NASA,Success,F9 v1.0,No attempt,2012-05-22,07:44:00
3,4,CCSFS SLC 40,SpaceX CRS-1,4700.0,LEO,NASA,Success,F9 v1.0,No attempt,2012-10-08,00:35:00
4,5,CCSFS SLC 40,SpaceX CRS-2,4877.0,LEO,NASA,Success,F9 v1.0,No attempt,2013-03-01,15:10:00


#### Location data of Launch sites is also important
Let's take them from SpaceX API dataset and append them in Wikipedia dataset

In [14]:
latitude = []
longitude = []

for launch_site in list(df_wiki['Launch site']):
    lon = df_api[df_api['LaunchSite'] == launch_site]['Longitude'].unique()[0]
    lat = df_api[df_api['LaunchSite'] == launch_site]['Latitude'].unique()[0]
    
    latitude.append(lat)
    longitude.append(lon)
    
df_wiki['Longitude'] = longitude
df_wiki['Latitude'] = latitude

In [15]:
df_wiki

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time,Longitude,Latitude
0,1,CCSFS SLC 40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,F9 v1.0,Failure,2010-06-04,18:45:00,-80.577366,28.561857
1,2,CCSFS SLC 40,SpaceX COTS Demo Flight 1,0.0,LEO,NASA,Success,F9 v1.0,Failure,2010-12-08,15:43:00,-80.577366,28.561857
2,3,CCSFS SLC 40,SpaceX COTS Demo Flight 2,525.0,LEO,NASA,Success,F9 v1.0,No attempt,2012-05-22,07:44:00,-80.577366,28.561857
3,4,CCSFS SLC 40,SpaceX CRS-1,4700.0,LEO,NASA,Success,F9 v1.0,No attempt,2012-10-08,00:35:00,-80.577366,28.561857
4,5,CCSFS SLC 40,SpaceX CRS-2,4877.0,LEO,NASA,Success,F9 v1.0,No attempt,2013-03-01,15:10:00,-80.577366,28.561857
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,239,CCSFS SLC 40,Starlink Group 5-15,0.0,LEO,SpaceX,Success,F9 B5,Success,2023-07-16,03:50:00,-80.577366,28.561857
239,240,VAFB SLC 4E,Starlink Group 6-15,0.0,LEO,SpaceX,Success,F9 B5,Success,2023-07-20,04:09:00,-120.610829,34.632093
240,241,CCSFS SLC 40,Starlink Group 6-6,0.0,LEO,SpaceX,Success,F9 B5,Success,2023-07-24,00:50:00,-80.577366,28.561857
241,242,CCSFS SLC 40,Starlink Group 6-7,0.0,LEO,SpaceX,Success,F9 B5,Success,2023-07-28,04:01:00,-80.577366,28.561857


In [16]:
df_wiki.dtypes

Flight No.           int64
Launch site         object
Payload             object
Payload mass       float64
Orbit               object
Customer            object
Launch outcome      object
Version Booster     object
Booster landing     object
Date                object
Time                object
Longitude          float64
Latitude           float64
dtype: object

In [17]:
df_wiki['Launch outcome'].value_counts()

Success    242
Failure      1
Name: Launch outcome, dtype: int64

In [18]:
df_wiki['Booster landing'].value_counts()

Success         200
No attempt       23
Failure          11
Controlled        6
Uncontrolled      2
Precluded         1
Name: Booster landing, dtype: int64

#### Update column names

In [19]:
col_names = list(df_wiki.columns)

for i, col_name in enumerate(col_names):
    col_names[i] = col_name.title().replace(' ', '_').strip('.')
    
col_names

['Flight_No',
 'Launch_Site',
 'Payload',
 'Payload_Mass',
 'Orbit',
 'Customer',
 'Launch_Outcome',
 'Version_Booster',
 'Booster_Landing',
 'Date',
 'Time',
 'Longitude',
 'Latitude']

In [20]:
df_wiki.columns = col_names

In [21]:
df_wiki.head()

,Flight_No,Launch_Site,Payload,Payload_Mass,Orbit,Customer,Launch_Outcome,Version_Booster,Booster_Landing,Date,Time,Longitude,Latitude
0,1,CCSFS SLC 40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,F9 v1.0,Failure,2010-06-04,18:45:00,-80.577366,28.561857
1,2,CCSFS SLC 40,SpaceX COTS Demo Flight 1,0.0,LEO,NASA,Success,F9 v1.0,Failure,2010-12-08,15:43:00,-80.577366,28.561857
2,3,CCSFS SLC 40,SpaceX COTS Demo Flight 2,525.0,LEO,NASA,Success,F9 v1.0,No attempt,2012-05-22,07:44:00,-80.577366,28.561857
3,4,CCSFS SLC 40,SpaceX CRS-1,4700.0,LEO,NASA,Success,F9 v1.0,No attempt,2012-10-08,00:35:00,-80.577366,28.561857
4,5,CCSFS SLC 40,SpaceX CRS-2,4877.0,LEO,NASA,Success,F9 v1.0,No attempt,2013-03-01,15:10:00,-80.577366,28.561857


#### Dealing with missing values

In [22]:
df_wiki.isna().sum()

Flight_No          0
Launch_Site        0
Payload            7
Payload_Mass       0
Orbit              2
Customer           1
Launch_Outcome     0
Version_Booster    0
Booster_Landing    0
Date               0
Time               0
Longitude          0
Latitude           0
dtype: int64

In [ ]:
df_wiki[df_wiki['Orbit'].isnull()]

,Flight_No,Launch_Site,Payload,Payload_Mass,Orbit,Customer,Launch_Outcome,Version_Booster,Booster_Landing,Date,Time,Longitude,Latitude
14,15,CCSFS SLC 40,DSCOVR,570.0,NaN,USAF,Success,F9 v1.1,Controlled,2015-02-11,23:03:00,-80.577366,28.561857
235,236,CCSFS SLC 40,Euclid,0.0,NaN,ESA,Success,F9 B5,Success,2023-07-01,15:12:00,-80.577366,28.561857


##### For this analysis the observation for missing Orbit value will be removed.

In [33]:
df_wiki.dropna(subset=['Orbit'], axis=0, inplace=True)

In [38]:
df_wiki.isna().sum()

Flight_No          0
Launch_Site        0
Payload            7
Payload_Mass       0
Orbit              0
Customer           1
Launch_Outcome     0
Version_Booster    0
Booster_Landing    0
Date               0
Time               0
Longitude          0
Latitude           0
dtype: int64

### Now dataset is ready to store in SQL database
Let's create final *csv* file and also store data in SQlite database for future use.

In [39]:
df_wiki.to_csv('falcon9_final.csv', index=False)

In [40]:
df_wiki['Booster_Landing'].unique()

array(['Failure', 'No attempt', 'Uncontrolled', 'Controlled', 'Precluded',
       'Success'], dtype=object)

#### SQlite database

In [41]:
con = sqlite3.connect("falcon9.db")
cur = con.cursor()

In [42]:
# save data in a SQlite databse table
df_wiki.to_sql("falcon9_tbl", con, if_exists='replace', index=False, method="multi")

241

Now data is in SQlite database.

- Please refer [SpaceX.ipynb](SpaceX.ipynb) Notebook file for the rest of the project.

#### Data preparation
- Please refer [Data_collection_spacex_api.ipynb](Data_collection_spacex_api.ipynb) for data collection SpaceX API.
- Please refer [Data_collection_Webscraping.ipynb](Data_collection_Webscraping.ipynb) for webscraping data from Falcon 9 Wikipedia page.
- Please refer [Complete_dataset.ipynb](Complete_dataset.ipynb) to see the implementatin of the final dataset.